# Online system testing

In [3]:
import numpy as np

from data_slicing.metadata import metadata
from data_slicing.slicing import get_data
from online_system.emd_ica_artifact_removal import get_imfs
from pprint import pprint
import importlib

## Read Data

In [4]:
patient_code = "p10"
patient_metadata = metadata[patient_code]
records = patient_metadata["records"]
record = records[0]
normal_record, seizure_record, info = get_data(record)

Extracting EDF parameters from D:\workspace2\eeg-epilepsy\data\EpilepticEEGDataset\p10_Record1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5401499  =      0.000 ... 10802.998 secs...
=== record_time === 2023-03-31 05:36:39
s_index, s_index_end:  3599500 3822000
=== raw_data.shape ===  (19, 5401500)
=== normal_record.shape ===  (19, 5179000)
=== seizure_record.shape ===  (19, 222500)


## Test EMD Decomposition

In [5]:
def get_imfs_it():
    test_data = normal_record[:, 0:2000]
    imfs, imf_ch_names = get_imfs(test_data, info["ch_names"])
    a = 1
    return imfs, imf_ch_names

imfs, imf_ch_names = get_imfs_it()

'artifact_removal, channel_number: 19 data_length: 2000'


In [6]:
pprint(imf_ch_names)

['Fp1-Ref imf1',
 'Fp1-Ref imf2',
 'Fp1-Ref imf3',
 'Fp1-Ref imf4',
 'Fp1-Ref imf5',
 'Fp1-Ref imf6',
 'Fp1-Ref imf7',
 'Fp2-Ref imf1',
 'Fp2-Ref imf2',
 'Fp2-Ref imf3',
 'Fp2-Ref imf4',
 'Fp2-Ref imf5',
 'Fp2-Ref imf6',
 'Fp2-Ref imf7',
 'Fp2-Ref imf8',
 'Fp2-Ref imf9',
 'F3-Ref imf1',
 'F3-Ref imf2',
 'F3-Ref imf3',
 'F3-Ref imf4',
 'F3-Ref imf5',
 'F3-Ref imf6',
 'F3-Ref imf7',
 'F3-Ref imf8',
 'F4-Ref imf1',
 'F4-Ref imf2',
 'F4-Ref imf3',
 'F4-Ref imf4',
 'F4-Ref imf5',
 'F4-Ref imf6',
 'F4-Ref imf7',
 'F4-Ref imf8',
 'C3-Ref imf1',
 'C3-Ref imf2',
 'C3-Ref imf3',
 'C3-Ref imf4',
 'C3-Ref imf5',
 'C3-Ref imf6',
 'C3-Ref imf7',
 'C3-Ref imf8',
 'C3-Ref imf9',
 'C4-Ref imf1',
 'C4-Ref imf2',
 'C4-Ref imf3',
 'C4-Ref imf4',
 'C4-Ref imf5',
 'C4-Ref imf6',
 'C4-Ref imf7',
 'C4-Ref imf8',
 'C4-Ref imf9',
 'P3-Ref imf1',
 'P3-Ref imf2',
 'P3-Ref imf3',
 'P3-Ref imf4',
 'P3-Ref imf5',
 'P3-Ref imf6',
 'P3-Ref imf7',
 'P3-Ref imf8',
 'P4-Ref imf1',
 'P4-Ref imf2',
 'P4-Ref imf3',
 'P4-Ref

In [7]:
from online_system.emd_ica_artifact_removal import imf_filtering

def imf_filtering_it():
    return imf_filtering(imfs, imf_ch_names)

eligible_imf_indexes = imf_filtering_it()
pprint(eligible_imf_indexes)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            144, 145, 146, 147, 148, 149, 152, 153, 154, 155],
           dtype='int64', length=119)


In [8]:
eligible_imfs = imfs[eligible_imf_indexes]
eligible_imf_ch_names = np.array(imf_ch_names)[eligible_imf_indexes]
print(eligible_imfs.shape)
print(eligible_imf_ch_names)

(119, 2000)
['Fp1-Ref imf1' 'Fp1-Ref imf2' 'Fp1-Ref imf3' 'Fp1-Ref imf4'
 'Fp1-Ref imf5' 'Fp1-Ref imf6' 'Fp1-Ref imf7' 'Fp2-Ref imf1'
 'Fp2-Ref imf2' 'Fp2-Ref imf3' 'Fp2-Ref imf4' 'Fp2-Ref imf7'
 'Fp2-Ref imf8' 'Fp2-Ref imf9' 'F3-Ref imf1' 'F3-Ref imf2' 'F3-Ref imf5'
 'F3-Ref imf6' 'F3-Ref imf7' 'F3-Ref imf8' 'F4-Ref imf1' 'F4-Ref imf3'
 'F4-Ref imf6' 'F4-Ref imf7' 'F4-Ref imf8' 'C3-Ref imf1' 'C3-Ref imf2'
 'C3-Ref imf6' 'C3-Ref imf7' 'C3-Ref imf8' 'C3-Ref imf9' 'C4-Ref imf1'
 'C4-Ref imf2' 'C4-Ref imf5' 'C4-Ref imf6' 'C4-Ref imf7' 'C4-Ref imf8'
 'C4-Ref imf9' 'P3-Ref imf4' 'P3-Ref imf5' 'P3-Ref imf6' 'P3-Ref imf7'
 'P3-Ref imf8' 'P4-Ref imf2' 'P4-Ref imf4' 'P4-Ref imf5' 'P4-Ref imf6'
 'P4-Ref imf7' 'P4-Ref imf8' 'O1-Ref imf1' 'O1-Ref imf2' 'O1-Ref imf3'
 'O1-Ref imf4' 'O1-Ref imf5' 'O1-Ref imf6' 'O1-Ref imf7' 'O1-Ref imf8'
 'O2-Ref imf1' 'O2-Ref imf5' 'O2-Ref imf6' 'O2-Ref imf7' 'O2-Ref imf8'
 'O2-Ref imf9' 'F7-Ref imf1' 'F7-Ref imf4' 'F7-Ref imf5' 'F7-Ref imf6'
 'F7-Ref imf7' 'F7-Ref

In [37]:
from online_system import emd_ica_artifact_removal
importlib.reload(emd_ica_artifact_removal)
ica_components_raw = emd_ica_artifact_removal.ICA_decompose(eligible_imfs, eligible_imf_ch_names)

Creating RawArray with float64 data, n_channels=119, n_times=2000
    Range : 0 ... 1999 =      0.000 ...     3.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 sec)

Fitting ICA to data using 119 channels (please be patient, this may take a while)
Selecting by number: 50 components


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 119 out of 119 | elapsed:    0.0s finished


Fitting ICA took 0.5s.


In [41]:
ica_components = np.array(ica_components_raw.get_data())
ica_components_filtering(ica_components, )

<class 'tuple'>
